In [2]:
%load_ext autoreload

In [27]:
%autoreload 2
import numpy as np
import pylab as plt

import odynn
from odynn.circuit import Circuit
from odynn.models import LeakyIntegrate, ChemSyn, GapJunction


from ipywidgets import interact

p = LeakyIntegrate.default_params
dt = 20.

@interact(g_gap = 0., g12=0.5, g21=0.5, mdp12=-40, mdp21=-40, scale12=10, scale21=10, E12=-80, E21=-80)
def dual(g_gap, g12, g21, mdp12, mdp21, scale12, scale21, E12, E21):
    connections = [(0,1) , (1,0)]
    t = np.arange(0.0, 5000., dt)
    i0 = 20. *((t>900)&(t<1050))
    i1 = 10.*((t>500)&(t<750)) #+ 20.*((t>700)&(t<750)) + 14*((t>1100)&(t<1800))+ 22*((t>1900)&(t<2000))
    i_injs = np.array([i0, i1]).transpose()
    
    neur = {'C_m': np.array([[1.], [1.]]), 
            'g_L': np.array([[0.5], [0.5]]), 
            'E_L': np.array([[-40.], [-40.]])}
    syns = {'G': np.array([[g12], [g21]]), 
            'mdp': np.array([[mdp12], [mdp21]]), 
            'scale': np.array([[scale12], [scale21]]), 
            'E': np.array([[E12], [E21]])}
    gap = {'G_gap' : np.array([[g_gap]])}
    n = LeakyIntegrate(init_p=neur)
    s = ChemSyn([c[0] for c in connections], [c[1] for c in connections], init_p = syns)
    g = GapJunction([0], [1], init_p=gap)
    circuit = Circuit(n, s, g)
    states = circuit.calculate(i_injs)
    plt.subplot(211)
    plt.plot(states[:,0,0,:,0])
    plt.subplot(212)
    plt.plot(i_injs)
    plt.show()
    

interactive(children=(FloatSlider(value=0.0, description='g_gap', max=1.0), FloatSlider(value=0.5, description…